In [1]:
from collections import defaultdict
import re

# our employees
employees = ["ianoneill", "helenkeller", "frankfruit", "carolonsloe", "bobmarley", "aishakhan"]

#mocked GitHub PR Data
github_data = [
    {
      "pull_request": {
        "title": "Add automated tests for auth service",
        "author": "ianoneill",
        "description": "Added 10+ test cases to increase coverage by 22%",
        "reviewers": [
          {
            "reviewer": "bobmarley",
            "comment": "This is fantastic coverage. Really nice job!",
            "sentiment": "positive"
          },
          {
            "reviewer": "helenkeller",
            "comment": "Left a few minor comments, but overall LGTM.",
            "sentiment": "neutral"
          }
        ],
        "merged_by": "carolonsloe",
        "acknowledged_contributor": "none"
      }
    },
    {
      "pull_request": {
        "title": "Fix flaky dashboard rendering bug",
        "author": "helenkeller",
        "description": "Resolved intermittent loading issue on dashboard page.",
        "reviewers": [
          {
            "reviewer": "frankfruit",
            "comment": "Thanks for fixing this — I hit this bug twice last week!",
            "sentiment": "positive"
          }
        ],
        "merged_by": "ianoneill",
        "acknowledged_contributor": "none"
      }
    },
    {
      "pull_request": {
        "title": "Update analytics pipeline for new metrics",
        "author": "bobmarley",
        "description": "Ingests new usage events and feeds them into the existing funnel structure.",
        "reviewers": [
          {
            "reviewer": "carolonsloe",
            "comment": "Big change — I left a few questions but overall looks good.",
            "sentiment": "neutral"
          }
        ],
        "merged_by": "frankfruit",
        "acknowledged_contributor": "aishakhan"
      }
    }
]

#transcript Data
transcript = [
    { "speaker": "ianoneill", "timestamp": "00:00:01", "text": "Morning, team! Quick sync today – mostly updates." },
    { "speaker": "carolonsloe", "timestamp": "00:00:05", "text": "We’ve been blocked on the API test cases again." },
    { "speaker": "bobmarley", "timestamp": "00:00:09", "text": "I’ll jump on that after this, Mark." },
    { "speaker": "ianoneill", "timestamp": "00:00:12", "text": "Thanks Bob, shoutout to Helen for the docs!" },
    { "speaker": "helenkeller", "timestamp": "00:00:16", "text": "No problem — happy to help!" },
    { "speaker": "carolonsloe", "timestamp": "00:00:20", "text": "Thanks Helen! And a huge shoutout to Ian for pushing that PR through." },
    { "speaker": "frankfruit", "timestamp": "00:00:24", "text": "Great job, everyone. Let's keep this momentum going." },
    { "speaker": "aishakhan", "timestamp": "00:00:28", "text": "I’ve been reviewing the analytics changes — looking solid so far." },
]

#Teams Chat Data
teams_chat = [
    {
      "sender": "bobmarley",
      "timestamp": "2025-04-08T13:20:00Z",
      "text": "Anyone have experience with the new Azure deployment config? Seeing weird errors."
    },
    {
      "sender": "ianoneill",
      "timestamp": "2025-04-08T13:21:30Z",
      "text": "Yeah, I ran into this last week. Try setting `enableDefaultIngress` to true."
    },
    {
      "sender": "helenkeller",
      "timestamp": "2025-04-08T13:23:10Z",
      "text": "That did the trick. Thanks, Ian!"
    },
    {
      "sender": "frankfruit",
      "timestamp": "2025-04-09T09:15:00Z",
      "text": "Need someone to sanity-check this SQL migration before I run it."
    },
    {
      "sender": "carolonsloe",
      "timestamp": "2025-04-09T09:17:00Z",
      "text": "On it. Looking now."
    },
    {
      "sender": "bobmarley",
      "timestamp": "2025-04-09T09:22:00Z",
      "text": "Looks clean, just double-check the rollback logic."
    },
    {
      "sender": "aishakhan",
      "timestamp": "2025-04-10T11:00:00Z",
      "text": "Huge thanks to Ian for unblocking the deployment flow. Quietly crushed it again 👏"
    }
]

#initialize scores for the six employees
impact_scores = defaultdict(lambda: {
    "helped_others": 0,
    "received_public_praise": 0,
    "unacknowledged_contribs": 0,
    "proactive_support": 0,
    "total_score": 0
})

# ---------- GITHUB DATA ANALYSIS ----------
for pr in github_data:
    pr_data = pr["pull_request"]
    author = pr_data["author"]
    reviewers = pr_data["reviewers"]
    acknowledged = pr_data.get("acknowledged_contributor")

    # PR authorship is a base contribution — not silent unless ignored
    # If acknowledged contributor is present and not author, give score
    if acknowledged and acknowledged != author:
        if acknowledged != "none":
            impact_scores[acknowledged]["unacknowledged_contribs"] += 1

    for review in reviewers:
        sentiment = review["sentiment"]
        reviewer = review["reviewer"]
        if sentiment == "positive":
            impact_scores[reviewer]["helped_others"] += 1
        elif sentiment == "neutral":
            impact_scores[reviewer]["helped_others"] += 0.5

# ---------- TRANSCRIPT ANALYSIS ----------
for entry in transcript:
    text = entry["text"]
    speaker = entry["speaker"]

    #look for praise or mentions by others
    for person in employees:
        if person != speaker and re.search(rf"\b{person}\b", text, re.IGNORECASE):
            if any(word in text.lower() for word in ["thanks", "shoutout", "helped", "fix", "great job", "nice work", "awesome"]):
                impact_scores[person]["received_public_praise"] += 1

# ---------- TEAMS CHAT ANALYSIS ----------
for message in teams_chat:
    sender = message["sender"]
    text = message["text"]

    for person in employees:
        if person != sender and re.search(rf"\b{person}\b", text, re.IGNORECASE):
            if "thanks" in text.lower() or "appreciate" in text.lower():
                impact_scores[person]["received_public_praise"] += 1

    #look for helpful replies (technical advice, support)
    if any(word in text.lower() for word in ["try", "suggest", "set", "fix", "looking", "checked", "reviewed"]):
        impact_scores[sender]["proactive_support"] += 1

# ---------- FINAL SCORING ----------
max_score = 0  # We'll find the maximum score for scaling

for person, metrics in impact_scores.items(): #weighting each category
    score = (
        1.5 * metrics["helped_others"] +
        2.0 * metrics["unacknowledged_contribs"] +
        1.2 * metrics["received_public_praise"] +
        1.5 * metrics["proactive_support"]
    )
    metrics["total_score"] = round(score, 2)
    max_score = max(max_score, metrics["total_score"])  # Keep track of the highest score

#scale scores to 100
for person, metrics in impact_scores.items():
    metrics["total_score"] = round((metrics["total_score"] / max_score) * 100, 2)

# ---------- OUTPUT RESULTS ----------
sorted_impact = sorted(impact_scores.items(), key=lambda x: x[1]['total_score'], reverse=True)

print("Silent Impact Rankings (Scaled to 100):\n")
for person, metrics in sorted_impact:
    print(f"{person}: {metrics}")


Silent Impact Rankings (Scaled to 100):

carolonsloe: {'helped_others': 0.5, 'received_public_praise': 0, 'unacknowledged_contribs': 0, 'proactive_support': 1, 'total_score': 100.0}
aishakhan: {'helped_others': 0, 'received_public_praise': 0, 'unacknowledged_contribs': 1, 'proactive_support': 0, 'total_score': 88.89}
bobmarley: {'helped_others': 1, 'received_public_praise': 0, 'unacknowledged_contribs': 0, 'proactive_support': 0, 'total_score': 66.67}
frankfruit: {'helped_others': 1, 'received_public_praise': 0, 'unacknowledged_contribs': 0, 'proactive_support': 0, 'total_score': 66.67}
ianoneill: {'helped_others': 0, 'received_public_praise': 0, 'unacknowledged_contribs': 0, 'proactive_support': 1, 'total_score': 66.67}
helenkeller: {'helped_others': 0.5, 'received_public_praise': 0, 'unacknowledged_contribs': 0, 'proactive_support': 0, 'total_score': 33.33}


In [2]:
pip install transformers torch


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 127.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 99.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 108.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitl

In [5]:
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

Device set to use cuda:0


In [7]:
# Function to get recent work for each employee
def get_recent_work(name, transcript, teams_chat, github_data):
    work = []

    # GitHub PRs
    for pr in github_data:
        pr_data = pr["pull_request"]
        if pr_data["author"] == name:
            work.append(f"{name} authored PR: {pr_data['title']} - {pr_data['description']}")

    # Transcript mentions
    for entry in transcript:
        if entry["speaker"] == name or name in entry["text"]:
            work.append(f"{name} said: {entry['text']}")

    # Teams chat mentions
    for message in teams_chat:
        if message["sender"] == name or name in message["text"]:
            work.append(f"{name} in Teams: {message['text']}")

    return work

# Function to get AI Insights for each employee
def get_ai_insights(name, transcript, teams_chat, github_data):
    recent_work = get_recent_work(name, transcript, teams_chat, github_data)
    full_text = f"Recent work for {name}:\n" + "\n".join(recent_work)

    # Generate insights using the summarizer
    insights = summarizer(full_text, max_length=50, min_length=20, do_sample=False)
    return insights[0]['summary_text']

# Get and print the results for each employee
for employee in employees:
    recent_work = get_recent_work(employee, transcript, teams_chat, github_data)
    ai_insights = get_ai_insights(employee, transcript, teams_chat, github_data)

    print(f"\nRecent Work for {employee}:\n")
    for work in recent_work:
        print(f"- {work}")

    print(f"\nAI Insights for {employee}:\n{ai_insights}")


Recent Work for ianoneill:

- ianoneill authored PR: Add automated tests for auth service - Added 10+ test cases to increase coverage by 22%
- ianoneill said: Morning, team! Quick sync today – mostly updates.
- ianoneill said: Thanks Bob, shoutout to Helen for the docs!
- ianoneill in Teams: Yeah, I ran into this last week. Try setting `enableDefaultIngress` to true.

AI Insights for ianoneill:
ianoneill said: Thanks Bob, shoutout to Helen for the docs! Try setting `enableDefaultIngress` to true.


Your max_length is set to 50, but your input_length is only 47. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=23)



Recent Work for helenkeller:

- helenkeller authored PR: Fix flaky dashboard rendering bug - Resolved intermittent loading issue on dashboard page.
- helenkeller said: No problem — happy to help!
- helenkeller in Teams: That did the trick. Thanks, Ian!

AI Insights for helenkeller:
Helenkeller said: No problem — happy to help! Ian said: That did the trick. Thanks, Ian!

Recent Work for frankfruit:

- frankfruit said: Great job, everyone. Let's keep this momentum going.
- frankfruit in Teams: Need someone to sanity-check this SQL migration before I run it.

AI Insights for frankfruit:
frankfruit said: Great job, everyone. Let's keep this momentum going. Need someone to sanity-check this SQL migration before I run it.


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset



Recent Work for carolonsloe:

- carolonsloe said: We’ve been blocked on the API test cases again.
- carolonsloe said: Thanks Helen! And a huge shoutout to Ian for pushing that PR through.
- carolonsloe in Teams: On it. Looking now.

AI Insights for carolonsloe:
Carolonsloe said: We’ve been blocked on the API test cases again.

Recent Work for bobmarley:

- bobmarley authored PR: Update analytics pipeline for new metrics - Ingests new usage events and feeds them into the existing funnel structure.
- bobmarley said: I’ll jump on that after this, Mark.
- bobmarley in Teams: Anyone have experience with the new Azure deployment config? Seeing weird errors.
- bobmarley in Teams: Looks clean, just double-check the rollback logic.

AI Insights for bobmarley:
bobmarley authored PR: Update analytics pipeline for new metrics. Ingests new usage events and feeds them into the existing funnel structure.

Recent Work for aishakhan:

- aishakhan said: I’ve been reviewing the analytics changes — looki